# Jupyter笔记本中的交互性问题

这个笔记本演示了Jupyter中使用widgets时的常见问题：
1. 需要手动设置回调函数
2. 需要手动重新运行单元格
3. 复杂的观察者模式

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np

## 问题1：需要手动设置回调

In [ ]:
# 创建滑块
slider = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='值:'
)

# 创建输出区域
output = widgets.Output()

def on_slider_change(change):
    with output:
        clear_output(wait=True)
        print(f"当前值: {change['new']}")
        print(f"平方: {change['new']**2}")

# 手动绑定回调函数
slider.observe(on_slider_change, names='value')

display(slider, output)

**问题**：
- 需要手动创建回调函数
- 需要手动管理输出区域
- 代码复杂且容易出错

## 问题2：多个widgets之间的同步困难

In [ ]:
# 创建多个相关的widgets
x_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Y:')
operation = widgets.Dropdown(
    options=['加法', '乘法', '幂运算'],
    value='加法',
    description='操作:'
)

result_output = widgets.Output()
plot_output = widgets.Output()

def update_calculation(change=None):
    x_val = x_slider.value
    y_val = y_slider.value
    op = operation.value
    
    if op == '加法':
        result = x_val + y_val
    elif op == '乘法':
        result = x_val * y_val
    else:  # 幂运算
        result = x_val ** y_val
    
    with result_output:
        clear_output(wait=True)
        print(f"{x_val} {op} {y_val} = {result:.2f}")
    
    # 更新图表
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6, 4))
        x_range = np.linspace(0.1, 5, 100)
        if op == '加法':
            y_range = x_range + y_val
        elif op == '乘法':
            y_range = x_range * y_val
        else:
            y_range = x_range ** y_val
        
        ax.plot(x_range, y_range)
        ax.axvline(x_val, color='red', linestyle='--', alpha=0.7)
        ax.set_xlabel('X')
        ax.set_ylabel('结果')
        ax.set_title(f'函数: f(x) = x {op} {y_val}')
        plt.show()

# 需要为每个widget手动绑定回调
x_slider.observe(update_calculation, names='value')
y_slider.observe(update_calculation, names='value')
operation.observe(update_calculation, names='value')

# 初始计算
update_calculation()

display(x_slider, y_slider, operation, result_output, plot_output)

**问题**：
- 需要为每个widget手动绑定相同的回调函数
- 代码重复且难以维护
- 容易忘记绑定某个widget
- 输出管理复杂

## 问题3：状态管理困难

In [ ]:
# 全局状态变量
current_data = None

# 数据生成控件
data_size = widgets.IntSlider(value=100, min=10, max=1000, description='数据量:')
noise_level = widgets.FloatSlider(value=0.1, min=0, max=1, description='噪声:')
generate_button = widgets.Button(description='生成数据')

# 分析控件
analysis_type = widgets.Dropdown(
    options=['均值', '标准差', '最大值', '最小值'],
    description='分析:'
)

data_output = widgets.Output()
analysis_output = widgets.Output()

def generate_data(button):
    global current_data
    size = data_size.value
    noise = noise_level.value
    
    # 生成数据
    current_data = np.random.normal(0, 1, size) + np.random.normal(0, noise, size)
    
    with data_output:
        clear_output(wait=True)
        print(f"已生成 {size} 个数据点，噪声水平: {noise}")
        plt.figure(figsize=(8, 4))
        plt.hist(current_data, bins=30, alpha=0.7)
        plt.title('数据分布')
        plt.show()
    
    # 触发分析更新
    update_analysis()

def update_analysis(change=None):
    global current_data
    if current_data is None:
        with analysis_output:
            clear_output(wait=True)
            print("请先生成数据")
        return
    
    analysis = analysis_type.value
    
    if analysis == '均值':
        result = np.mean(current_data)
    elif analysis == '标准差':
        result = np.std(current_data)
    elif analysis == '最大值':
        result = np.max(current_data)
    else:  # 最小值
        result = np.min(current_data)
    
    with analysis_output:
        clear_output(wait=True)
        print(f"{analysis}: {result:.4f}")

generate_button.on_click(generate_data)
analysis_type.observe(update_analysis, names='value')

display(data_size, noise_level, generate_button, data_output)
display(analysis_type, analysis_output)

**问题**：
- 需要使用全局变量管理状态
- 手动管理widget之间的依赖关系
- 容易出现状态不一致的问题
- 代码结构复杂，难以理解和维护